In [18]:
import sys
import os

In [20]:
repo_path = r"C:\Users\rnhhy\turkish-parliament-texts"
sys.path.append(repo_path)

In [22]:
import corpus_loader
import configparser
from corpus_compiler import tbmmcorpus

In [24]:
import configparser
from corpus_compiler import tbmmcorpus
import logging

config_parser = configparser.ConfigParser()

config_file_path = r"C:\Users\rnhhy\turkish-parliament-texts\config.ini" 
config_parser.read(config_file_path)

if 'default' in config_parser:
    config = config_parser['default']
else:
    raise KeyError("The 'default' section is not found in the configuration file.")

corpus_filepath = r"C:\Users\rnhhy\corpus-v0.4b\tbmm_corpus.mm"
date_mappings_path = r"C:\Users\rnhhy\turkish-parliament-texts\data\date_mappings.pkl"

if not os.path.isfile(corpus_filepath):
    raise FileNotFoundError(f"The corpus file {corpus_filepath} does not exist.")
if not os.path.isfile(date_mappings_path):
    raise FileNotFoundError(f"The date mappings file {date_mappings_path} does not exist.")

corpus = tbmmcorpus.TbmmCorpus(metadata=True, config=config)

corpus.load_tbmm_corpus(corpus_filepath, inmemory=True)

print("Corpus loaded successfully with inmemory=True")

INFO:corpus_compiler.tbmmcorpus:loaded 100 documents
INFO:corpus_compiler.tbmmcorpus:loaded 200 documents
INFO:corpus_compiler.tbmmcorpus:loaded 300 documents
INFO:corpus_compiler.tbmmcorpus:loaded 400 documents
INFO:corpus_compiler.tbmmcorpus:loaded 500 documents
INFO:corpus_compiler.tbmmcorpus:loaded 600 documents
INFO:corpus_compiler.tbmmcorpus:loaded 700 documents
INFO:corpus_compiler.tbmmcorpus:loaded 800 documents
INFO:corpus_compiler.tbmmcorpus:loaded 900 documents
INFO:corpus_compiler.tbmmcorpus:loaded 1000 documents
INFO:corpus_compiler.tbmmcorpus:loaded 1100 documents
INFO:corpus_compiler.tbmmcorpus:loaded 1200 documents
INFO:corpus_compiler.tbmmcorpus:loaded 1300 documents
INFO:corpus_compiler.tbmmcorpus:loaded 1400 documents
INFO:corpus_compiler.tbmmcorpus:loaded 1500 documents
INFO:corpus_compiler.tbmmcorpus:loaded 1600 documents
INFO:corpus_compiler.tbmmcorpus:loaded 1700 documents
INFO:corpus_compiler.tbmmcorpus:loaded 1800 documents
INFO:corpus_compiler.tbmmcorpus:loade

Corpus loaded successfully with inmemory=True


In [26]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_val_score
from sklearn.decomposition import IncrementalPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from scipy import stats
import time
import pandas as pd
import numpy as np

In [28]:
data = corpus

In [30]:
def fit_in_batches(classifier, x_train, y_train, batch_size):
    if hasattr(classifier, 'partial_fit'):
        for i in range(0, len(x_train), batch_size):
            x_batch = x_train[i:i + batch_size]
            y_batch = y_train[i:i + batch_size]
            classifier.partial_fit(x_batch, y_batch, classes=np.unique(y_train))
    else:
        classifier.fit(x_train, y_train)
    return classifier

In [32]:
def pre_steps(data, vectorizer, compressor, classifier):

    fold_matrix = []
    kf = KFold(n_splits=10, random_state=42, shuffle=True)
    data_vector = vectorizer.fit_transform(data.data).astype('float32').toarray()

    if compressor is not None:
        if isinstance(compressor, LDA):
            data_vector = compressor.fit_transform(data_vector, data.target)
        else:
            data_vector = compressor.fit_transform(data_vector)

    for i, (train_index, test_index) in enumerate(kf.split(data_vector, data.target)):

        data_train = data_vector[train_index]
        data_test = data_vector[test_index]
        data_train_target = data.target[train_index]
        data_test_target = data.target[test_index]
        data_train_target = np.array(data_train_target)
        data_test_target = np.array(data_test_target)
        
        fitted_classifier = fit_in_batches(classifier, data_train, data_train_target, batch_size=500)
        fold_matrix.append([data_train, data_test, data_train_target, data_test_target, fitted_classifier])
   
    return fold_matrix

In [34]:
# Vectorizer and compressor combinations
vectorizer = [CountVectorizer(), TfidfVectorizer()]
compressor = [None, IncrementalPCA(n_components=50, batch_size=1000), LDA(n_components=2)]
classifier = [SGDClassifier(loss="hinge", penalty="l2", alpha=1e-3, random_state=42, max_iter=5, tol=None), LogisticRegression(max_iter=1000), 
              SVC(kernel='linear', random_state=42)]
combinations = [(vec, comp, classify) for vec in vectorizer for comp in compressor for classify in classifier]

# Re-initialize classifier for each combination
all_results = []
execution_times = []
for vec, comp, classify in combinations:
    start_time = time.time()
    if isinstance(classify, SGDClassifier):
        classifier_instance = SGDClassifier(loss="hinge", penalty="l2", alpha=1e-3, random_state=42, max_iter=5, tol=None)
    elif isinstance(classify, LogisticRegression):
        classifier_instance = LogisticRegression(max_iter=1000)
    elif isinstance(classify, SVC):
        classifier_instance = SVC(kernel='linear', random_state=42)
    
    result = pre_steps(data, vec, comp, classifier_instance)
    end_time = time.time()
    execution_time = end_time - start_time
    execution_times.append(execution_time)
    print(f"Evaluating Model with {vec.__class__.__name__}, {comp.__class__.__name__ if comp else 'None'}, {classify.__class__.__name__}: {execution_time: .6f} seconds")
    all_results.append(result)

AttributeError: 'TbmmCorpus' object has no attribute 'data'